In [1]:
import requests
import json
import re

from collections import Counter

from homeworks.utils.aidevsApi import get_auth_token, get_task, send_answer, API_KEY_OPENAI
from openai import OpenAI

client = OpenAI(api_key=API_KEY_OPENAI)

token = get_auth_token('md2html')
task = get_task(token, print_task=True)

----------- Task description -----------
msg: Provide me the URL to your API (HTTPS) via /answer/ endpoint. I will send you some Markdown formated text, and your task is to convert it to HTML. Please fine-tuning model for that.
hint1: Probably I will ask more than one question, so be prepared for that
hint2: Please return the answer in JSON format, with "reply" field!
hint3: Do NOT add any additional new lines to the output! If sentence is just in one line, then don't break it to 2-3
-----------    ----------    -----------


## Fine-tuning GTP-3-5
used https://platform.openai.com/playground?mode=chat&model=gpt-4-1106-preview
to generate examples with this system prompt:

Your role is to generate example sentences or short paragraphs of JSONL data designed for fine-tuning a model to convert Markdown (MD) to HTML.

Please focus on H1 , H2, H3, bold, italics, underline, citation and links.

In Markdown use this:
- underline _ ,
- bold: **,
- italic: *
- blockquotes: <

Give so many examples how many user will ask.

Return examples in this format as messages to ChatGPT:
User: 1
Assistant: {"messages":[{"role":"system","content":"md2html"},{"role":"user","content":"this is **important** note about [AI Devs](https://aidevs.pl). Please _read_ it!"},{"role":"assistant","content":"this is <b>important</b> note about <a href=\"https://aidevs.pl\">AI Devs</a>. Please <u>read</u> it!"}]}

#### I manualy imported file and fine-tuned model on playground
https://platform.openai.com/finetune/

In [2]:
# testing fine-tuned model

md_example = "this is **important** note about [AI Devs](https://aidevs.pl). Please _read_ it!"

response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-1106:mirek::8RQSzYwt",
  messages=[
    {"role": "system", "content": "Change user Markdown to HTML"},
    {"role": "user", "content": md_example}
  ]
)

print(response.choices[0].message.content)

this is <b>important</b> note about <a href="https://aidevs.pl">AI Devs</a>. Please <u>read</u> it!


### Test own API

In [3]:
# testing my temporary API:
url = "https://myownapiv41.azurewebsites.net/md2html/"

data = {
    "question": md_example
}
headers = {
    "Content-Type": "application/json"
}

data_json = json.dumps(data)

response = requests.post(url, data=data_json, headers=headers)

response.json()

{'reply': 'this is <b>important</b> note about <a href="https://aidevs.pl">AI Devs</a>. Please <u>read</u> it!'}

### Sending answer

In [4]:
token = get_auth_token('md2html')
answer = url
send_answer(token, answer)

Sending answer: done! :)


#### Additional code used for API :)

In [ ]:
def mk2html(md_example):
    # ask LLM compressing
    response = client.chat.completions.create(
      model="ft:gpt-3.5-turbo-1106:mirek::8RQSzYwt",
      messages=[
        {"role": "user", "content": md_example}
      ]
    )
    
    q = response.choices[0].message.content
    
    return q